## Installation Of Gensim For Word2Vec

In [1]:
conda install -c gensim


Note: you may need to restart the kernel to use updated packages.


In [2]:
from gensim.models import Word2Vec,KeyedVectors

## Import Sklearn

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

## Importing Dataset

In [4]:
df= pd.read_csv('data_scraped6.csv')

## Trying For Combination

In [5]:
Combo = df["Body"]+df["Comments"]
df = df.assign(Combo = Combo) 
df['Combo']=df['Combo'].fillna("")

In [6]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

## Taking the values from the dataset

In [7]:
Body=df['Combo'].values.tolist()

## Cleaning of the words by tokenising them and removing punctuations and stop words

In [8]:
newVec=list()
for line in Body:
    #create words token as well as remove punctuation in one go
    rem_tok_punc=RegexpTokenizer(r'\w+')
    tokens=rem_tok_punc.tokenize(line)
    
    #convert words to lower
    words=[w.lower() for w in tokens]
    #Invoke all english stopwords
    stop_word_list=list(stopwords.words('english'))
    stop_word_list.append('https')
    stop_word_list.append('www')
    stop_word_list.append('.com')
    #Remove Stop words
    words=[w for w in words if not w in stop_word_list]
    #Append words in the newVec
    newVec.append(words)
len(newVec)

4985

## Fitting Word2Vector Model 

In [9]:
model=Word2Vec(newVec,min_count=4,size=100,window=10,sg=1,iter=10)

## Number Of Words in Vocabulary

In [10]:
# This will give the total number of words in the vocabolary created from this dataset
model.wv.syn0.shape

C:\Users\OASIS\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


(25030, 100)

## Finding Vector Of Each Word

In [11]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    ##print(index2word_set)
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [12]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [13]:
# Calculating average feature vector for training set
clean_train_reviews=df['Combo'].values
num_features=100
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

Review 0 of 4985


C:\Users\OASIS\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
C:\Users\OASIS\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


Review 1000 of 4985
Review 2000 of 4985
Review 3000 of 4985
Review 4000 of 4985


In [14]:
trainDataVecs.shape

(4985, 100)

In [15]:
df.shape

(4985, 14)

In [16]:
data=trainDataVecs
traindata=pd.DataFrame(data=data[0:,0:])  # 1st row as the column names
traindata.shape

(4985, 100)

In [17]:
traindata=traindata.fillna(0)
traindata.describe

<bound method NDFrame.describe of             0         1         2         3         4         5         6   \
0     0.137557 -0.304873 -0.098512 -0.042960  0.148453  0.081600  0.280950   
1     0.144671 -0.318159 -0.075927 -0.007181  0.129896  0.084479  0.280024   
2     0.122943 -0.306415 -0.090790 -0.040546  0.134035  0.097555  0.276394   
3     0.150402 -0.336301 -0.070564  0.000638  0.165161  0.098744  0.251839   
4     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
4980  0.138858 -0.319479 -0.103362 -0.018939  0.148225  0.080729  0.309365   
4981  0.145783 -0.335899 -0.104094  0.010063  0.144384  0.072581  0.243057   
4982  0.116315 -0.369161 -0.104393  0.021386  0.176573  0.009036  0.218449   
4983  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4984  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

            7         8      

## Various Algorithms(LogisticRegression, KNN, CART,RandomForest,NaiveBayes) 

In [18]:
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('LR',LogisticRegression(max_iter=500,solver='lbfgs',multi_class='multinomial')))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('Rand_Forest', RandomForestClassifier()))
models.append(('NB', BernoulliNB()))
models.append (('SDG', SGDClassifier()))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=seed,shuffle=True)
	cv_results = model_selection.cross_val_score(model, traindata,df["Flair"], cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

LR: 0.218259 (0.020074)
KNN: 0.191378 (0.030163)
CART: 0.519571 (0.026247)


C:\Users\OASIS\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\OASIS\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\OASIS\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\OASIS\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\OASIS\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

Rand_Forest: 0.517569 (0.026909)
NB: 0.157878 (0.016715)
SDG: 0.181531 (0.023294)
